# Viszualizing Distances on a world map #

The information about the locations can be found in the folder "data". For the aim of the project, we differntiate between datacenters (hosts) and users (clients). They're saved in csv-files containing the main information. 

In [1]:
import pandas as pd
import numpy as np
import folium
import folium.plugins
import geo_convert
from shapely.geometry import LineString
import geopandas as gpd

The data is imported into pandas dataframe.

In [2]:
hosts_df = pd.read_csv("data/hosts.csv", dtype={"Name": str, "land": str, "street": str, "number": str, "zip": str, "city": str})
clients_df = pd.read_csv("data/clients.csv", dtype={"Name": str, "land": str, "street": str, "number": str, "zip": str, "city": str})
if "longitude" not in list(hosts_df):
    host_coord = []
    for i, v in hosts_df.iterrows():
        address = {"street": v["street"],
                   "number": v["number"],
                   "city": v["city"],
                   "country": v["land"],
                   "zip_code": v["zip"]}
        host_coord.append(geo_convert.geocode_address(address))
    hosts_df["latitude"] = pd.Series([i[0] for i in host_coord])
    hosts_df["longitude"] = pd.Series([i[1] for i in host_coord])
    hosts_df.to_csv("data/hosts.csv", index=False, encoding="utf8")
if "longitude" not in list(clients_df):
    client_coord = []
    for i, v in clients_df.iterrows():
        address = {"street": v["street"],
                   "number": v["number"],
                   "city": v["city"],
                   "country": v["land"],
                   "zip_code": v["zip"]}
        client_coord.append(geo_convert.geocode_address(address))
    clients_df["latitude"] = pd.Series([i[0] for i in client_coord])
    clients_df["longitude"] = pd.Series([i[1] for i in client_coord])
    clients_df.to_csv("data/clients.csv", index=False, encoding="utf8")
print(clients_df)
print(hosts_df)

            name     land          street number    zip      city  \
0  Suomen Pankki  Finland  Snellmaninkatu      2  00170  Helsinki   

           notes   latitude  longitude  
0  Finlands bank  60.170329  24.953266  
  name     land           street number    zip       city              notes  \
0  EZB  Germany  Sonnemannstraße     22  60314  Frankfurt  Adress of the EZB   

   latitude  longitude  
0  50.11133   8.704547  


We have now imported the data and looked up the geo-data. In the folowing part, the informations will be visualized on a map.

In [3]:
map_folium = folium.Map(location=[48.8566969, 2.3514616], zoom_start=2)
for i, v in hosts_df.iterrows():
    folium.Marker([v["latitude"], v["longitude"]],
                  popup="<h3>%s</h3>%s" % (v["name"], v["notes"]),
                  icon = folium.Icon(color="lightgray", icon="cloud", prefix="fa")).add_to(map_folium)
lines = []
for i, v in clients_df.iterrows():
    folium.Marker([v["latitude"], v["longitude"]],
                  popup="<h3>%s</h3>%s" % (v["name"], v["notes"]),
                  icon = folium.Icon(color="red", icon="user", prefix="fa")).add_to(map_folium)
    for _, host in hosts_df.iterrows():
        lines.append(LineString([(v["longitude"], v["latitude"]), 
                                (host["longitude"], host["latitude"])])
                    )
    
folium.plugins.Fullscreen().add_to(map_folium)
line_gdf = gpd.GeoDataFrame(geometry=[l for l in lines])

for _, r in line_gdf.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    point1 = sim_geo[0].xy[0]
    point2 = sim_geo[0].xy[1]
    distance = np.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup("Distance: %s" % distance).add_to(geo_j)
    geo_j.add_to(map_folium)

map_folium